In [1]:
from selenium import webdriver
from pymongo import MongoClient
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common import exceptions
import time

In [2]:
MONGO_URI = 'mongodb://172.17.0.2:27017/'
MONGO_DATABASE = 'mvideo_db'

client = MongoClient(MONGO_URI)
mongo_base = client[MONGO_DATABASE]
collection = mongo_base['bestsellers']

In [3]:
chrome_options = Options()
chrome_options.add_argument('start-maximized')


driver = webdriver.Chrome(options=chrome_options)

url = 'https://www.mvideo.ru'
title_site = 'М.Видео'

driver.get(url)

# driver.title

In [ ]:
time.sleep(5)
assert title_site in driver.title

try:
    bestsellers = driver.find_element_by_xpath(
        '//div[contains(text(),"Хиты продаж")]/ancestor::div[@data-init="gtm-push-products"]'
    )
except exceptions.NoSuchElementException:
    print('Bestsellers has not been found')
# print(bestsellers)

<selenium.webdriver.remote.webelement.WebElement (session="6fd93ebd587c384c5fa4aae1f8134dc3", element="96ab0401-11af-40ea-b0da-bae222c40951")>


In [ ]:
time.sleep(15)
while True:
    try:
        next_button = bestsellers.find_element_by_class_name('next-btn')
        next_button.click()
        time.sleep(1)
    except exceptions.TimeoutException:
        print('time out')
        break

In [ ]:
goods = bestsellers.find_elements_by_css_selector('li.gallery-list-item')
goods

In [ ]:

goods = bestsellers.find_elements_by_css_selector('li.gallery-list-item')


item = {}
for good in goods:    
    item['link'] = good.find_element_by_css_selector(
        'a.sel-product-tile-title') \
        .get_attribute('href')
    
    item['title'] = good.find_element_by_css_selector(
        'a.sel-product-tile-title') \
        .get_attribute('innerHTML')

    item['price'] = float(
        good.find_element_by_css_selector(
            'div.c-pdp-price__current').get_attribute('innerHTML').replace(
                '&nbsp;', '').replace('¤', ''))

    item['image_link'] = good.find_element_by_css_selector(
        'img[class="lazy product-tile-picture__image"]') \
        .get_attribute('src')
    print(item)

    collection.update_one({'link': item['link']}, {'$set': item},
                          upsert=True)
driver.quit()
